# wps_full_rvic

#### wps_full_rvic is a process that runs the [parameters](https://rvic.readthedocs.io/en/latest/user-guide/parameters/) and [convolution](https://rvic.readthedocs.io/en/latest/user-guide/convolution/) module of a VIC streamflow routing model, called [RVIC](https://rvic.readthedocs.io/en/latest/). To get started, first instatiate the client. Here, the client will try to connect to a remote osprey instance using the url parameter.

In [ ]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools import copy_http_content

# Ensure we are in the working directory with access to the data
if os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../../../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [ ]:
# NBVAL_IGNORE_OUTPUT
osprey.full_rvic?

#### We can use the docstring to ensure we provide the appropriate parameters.

In [ ]:
# run full_rvic
output = osprey.full_rvic(
    params_config = "tests/data/configs/parameters_local.cfg",
    convolve_config = "tests/data/configs/convolve_opendap.cfg"
)

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [ ]:
output_data = Dataset(make_tmp_copy(output))

# Metadata of the final output of RVIC parameters + convolution 
metadata = {
    "title": output_data.title,
    "year": output_data.history.split()[5],
    "month": output_data.history.split()[2],
    "day": output_data.history.split()[3],
    "RvicPourPointsFile": output_data.RvicPourPointsFile,
    "RvicUHFile": output_data.RvicUHFile,
    "RvicFdrFile": output_data.RvicFdrFile,
    "RvicDomainFile": output_data.RvicDomainFile,
}

In [ ]:
input_config = read_config("tests/data/configs/parameters_opendap.cfg")

# Expected output metadata determined from the input configuration file
expected = {
    "title": "RVIC history file",
    "year": datetime.now().strftime("%Y"),
    "month": datetime.now().strftime("%b"),
    "day": datetime.now().strftime("%d"),
    "RvicPourPointsFile": input_config["POUR_POINTS"]["FILE_NAME"].split("/")[-1],
    "RvicUHFile": input_config["UH_BOX"]["FILE_NAME"].split("/")[-1],
    "RvicFdrFile": input_config["ROUTING"]["FILE_NAME"].split("/")[-1],
    "RvicDomainFile": input_config["DOMAIN"]["FILE_NAME"].split("/")[-1],
}

assert metadata == expected